# Usage of Dataset class
We here show the API of the Dataset class, conceived to run and collects ensemble calculations.

In [1]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I

In [2]:
from BigDFT.Database import Molecules
CO=Molecules.Molecule('CO')
print CO

{'units': 'angstroem', 'positions': [{'C': [0.0, 0.0, 0.0], 'sym': 'C'}, {'sym': 'O', 'O': [0.0, 0.0, 1.1282]}], 'global monopole': 0.0}


In [3]:
inp=I.Inputfile({'dft': {'rmult': [3,8]}})

In [4]:
code=C.SystemCalculator(omp=2,mpi_run='mpirun -np 2')

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 2 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft


In [5]:
reload(D)

<module 'BigDFT.Datasets' from '/home/marco/Applications/BigDFT/binaries/v1.8.3/install/lib/python2.7/site-packages/BigDFT/Datasets.pyc'>

In [6]:
study=D.Dataset(label='CO',run_dir='CO',posinp=CO)

In [7]:
inp.set_rmult(coarse=3)
study.append_run(id={'crmult': 3}, runner= code, input=inp,skip=False)
inp.set_rmult(coarse=4)
study.append_run(id={'crmult': 4}, runner= code, input=inp)
inp.set_rmult(coarse=5)
study.append_run(id={'crmult': 5}, runner= code, input=inp)

In [8]:
code.update_global_options(skip=False)

In [9]:
data=study.run()

Creating the yaml input file "CO/crmult:3.yaml"
Run directory CO
Executing command:  mpirun -np 2 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n crmult:3
Creating the yaml input file "CO/crmult:4.yaml"
Run directory CO
Executing command:  mpirun -np 2 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n crmult:4
Creating the yaml input file "CO/crmult:5.yaml"
Run directory CO
Executing command:  mpirun -np 2 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n crmult:5


In [10]:
study.fetch_results(attribute='energy')

[-21.611635946132814, -21.64835568408899, -21.651311269276533]

In [11]:
for d in range(len(study.runs)):
    print study.runs[d]
    print study.results[d].energy

{'run_dir': 'CO', 'input': {'dft': {'rmult': [3, 8.0]}}, 'posinp': {'units': 'angstroem', 'positions': [{'C': [0.0, 0.0, 0.0], 'sym': 'C'}, {'O': [0.0, 0.0, 1.1282], 'sym': 'O'}], 'global monopole': 0.0}, 'skip': False, 'label': 'CO'}
-21.6116359461
{'run_dir': 'CO', 'input': {'dft': {'rmult': [4, 8.0]}}, 'posinp': {'units': 'angstroem', 'positions': [{'C': [0.0, 0.0, 0.0], 'sym': 'C'}, {'O': [0.0, 0.0, 1.1282], 'sym': 'O'}], 'global monopole': 0.0}, 'label': 'CO'}
-21.6483556841
{'run_dir': 'CO', 'input': {'dft': {'rmult': [5, 8.0]}}, 'posinp': {'units': 'angstroem', 'positions': [{'C': [0.0, 0.0, 0.0], 'sym': 'C'}, {'O': [0.0, 0.0, 1.1282], 'sym': 'O'}], 'global monopole': 0.0}, 'label': 'CO'}
-21.6513112693


# Polarizability tensor of a molecule

In [87]:
inp=I.Inputfile()
mol=Molecules.Molecule('CO')
#mol['positions'][1]['O']=[0.0,0.0,1.12016] #our positions
inp.set_xc('PBE')
inp.set_hgrid('0.37')
inp.set_rmult(coarse=11)
logGS=code.run(input=inp,posinp=mol,name='CO-GS')

Creating the yaml input file "./CO-GS.yaml"
Executing command:  mpirun -np 2 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n CO-GS


In [88]:
import numpy as np
intensity=-1.e-2
PolTensor=D.Dataset(label='Poltensor',run_dir='polt',input=inp,posinp=mol,d0=logGS.dipole,F=intensity)
for idir,coord in enumerate(['x','y','z']):
    el=np.zeros(3)
    el[idir]=intensity
    inp.apply_electric_field(el.tolist())
    PolTensor.append_run({'id': coord,'F':intensity},code,input=inp)

In [89]:
def extract_alpha(PolT):
    d0=np.array(PolT.get_global_option('d0'))
    F=PolT.get_global_option('F')
    ds=PolT.fetch_results(attribute='dipole')
    alpha=np.mat(np.zeros(9)).reshape(3,3)
    for idir in range(3):
        alpha[idir]=(np.array(ds[idir])-d0)/F
    return alpha

In [90]:
code.update_global_options(skip=False)

In [91]:
PolTensor.set_postprocessing_function(extract_alpha)

In [92]:
alpha=PolTensor.run()

Creating the yaml input file "polt/F:-0.01,id:x.yaml"
Run directory polt
Executing command:  mpirun -np 2 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:-0.01,id:x
Creating the yaml input file "polt/F:-0.01,id:y.yaml"
Run directory polt
Executing command:  mpirun -np 2 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:-0.01,id:y
Creating the yaml input file "polt/F:-0.01,id:z.yaml"
Run directory polt
Executing command:  mpirun -np 2 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:-0.01,id:z


In [50]:
alpha1=_

In [51]:
alpha1

matrix([[ 1.26423306e+01,  3.50890610e-03, -3.31700000e-02],
        [ 3.50890610e-03,  1.26423306e+01, -3.31700000e-02],
        [ 3.70690000e-05,  3.70690000e-05,  1.58543700e+01]])

In [64]:
alpha2=alpha

In [65]:
alpha1

matrix([[ 1.26423306e+01,  3.50890610e-03, -3.31700000e-02],
        [ 3.50890610e-03,  1.26423306e+01, -3.31700000e-02],
        [ 3.70690000e-05,  3.70690000e-05,  1.58543700e+01]])

In [57]:
alpha2

{0: <BigDFT.Logfiles.Logfile instance at 0x7f02c026f950>,
 1: <BigDFT.Logfiles.Logfile instance at 0x7f02c026f7a0>,
 2: <BigDFT.Logfiles.Logfile instance at 0x7f0291a53128>}

In [85]:
alpha

matrix([[ 1.26686738e+01, -3.23449840e-03,  3.77370000e-02],
        [-3.23449840e-03,  1.26686738e+01,  3.77370000e-02],
        [-1.11158900e-03, -1.11158900e-03,  1.58224360e+01]])

In [68]:
a=0.5*(alpha1+alpha2)

In [93]:
np.mean(np.diag(alpha))*(0.529177**3)

2.1334035481704943